# Testing on the qra-biodegradable dataset


In [1]:
# imports
# make AntClus dir known
import sys

sys.path.append("../AntClust")

import sys

# import matplotlib.pyplot as plt
import time as time

import numpy as np
import pandas as pd
from AntClust import AntClust
from distance_classes import similarity_1d
from rules import labroche_rules
from scipy.io import arff

In [2]:
def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min())

In [3]:
# har testing set

# make set where last column=label is removed
# transpose the whole thing and find max per feature
# make similarity func for every feature and save it to array
# run AntClust
# compare results with the last item from the vector
# for frame_index in range(len(data)):

In [4]:
# -----------------------------------
#        load dataset
# -----------------------------------
data = arff.loadarff("datasets/qsar-biodeg.arff")

# load data as pandas data frame
df = pd.DataFrame(data[0])

# save labels
labels = df[df.columns[len(df.columns) - 1]].astype(int).to_numpy()

# drop labels
df.drop(df.columns[len(df.columns) - 1], axis=1, inplace=True)

# normalize it via min_max_scaling
for col in df.columns:
    df[col] = min_max_scaling(df[col])

# or as a one liner
# normalized_df = (df-df.min())/(df.max()-df.min())

# save it as numpy array
data_np = df.to_numpy()

In [5]:
df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41
0,0.426824,0.225351,0.000000,0.000000,0.00,0.000000,0.000000,0.517298,0.083333,0.000000,...,0.00,0.000000,0.000000,0.000000,0.080921,0.028453,0.000,0.238782,0.00,0.000000
1,0.482651,0.156504,0.000000,0.000000,0.00,0.000000,0.000000,0.507414,0.041667,0.083333,...,0.00,0.000000,0.000000,0.000000,0.124347,0.114732,0.000,0.239190,0.00,0.000000
2,0.429715,0.292264,0.000000,0.000000,0.00,0.000000,0.000000,0.439868,0.083333,0.333333,...,0.00,0.000000,0.000000,0.142857,0.095990,0.217990,0.000,0.274353,0.00,0.000000
3,0.222420,0.227608,0.000000,0.000000,0.00,0.000000,0.000000,0.329489,0.000000,0.166667,...,0.00,0.000000,0.000000,0.142857,0.092430,0.810693,0.000,0.181233,0.00,0.000000
4,0.497331,0.309365,0.000000,0.000000,0.00,0.000000,0.000000,0.484349,0.083333,0.333333,...,0.00,0.000000,0.000000,0.000000,0.128619,0.215236,0.000,0.315445,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,0.763123,0.249785,0.000000,0.000000,0.00,0.153846,0.000000,0.528830,0.166667,0.083333,...,0.25,0.000000,0.333333,0.142857,0.154960,0.177834,0.125,0.324134,0.00,0.000000
1051,0.731094,0.306833,0.000000,0.000000,0.25,0.000000,0.000000,0.581549,0.000000,0.750000,...,0.00,0.000000,0.166667,0.000000,0.180351,0.370812,0.375,0.445773,0.00,0.000000
1052,0.638123,0.115016,0.000000,0.333333,0.25,0.000000,0.277778,0.731466,0.000000,0.333333,...,0.00,0.333333,0.722222,0.000000,0.187589,0.254475,0.625,0.472248,0.25,0.000000
1053,0.702402,0.105988,0.166667,0.000000,1.00,0.000000,0.500000,0.924217,0.000000,0.000000,...,0.00,0.083333,0.888889,0.000000,0.420147,0.134924,1.000,0.627415,0.00,0.037037


# AntClust


In [6]:
# normal numeric Distance, dataset is normalized so we know min = 0 max = 1
min = 0
max = 1
sim_numeric = similarity_1d(min, max)

num_features = len(data_np[0])
print("making similarity function vector")
similarity_per_feature = [sim_numeric] * num_features

# -----------------------------------
#       define ruleset
# -----------------------------------
ruleset = labroche_rules()

# -----------------------------------
#       define AntClust
# -----------------------------------
print("Make Ant Colony")
ant_clust = AntClust(
    data_np,
    similarity_per_feature,
    ruleset,
    store_computed_similaritys=True,
    alpha_ant_meeting_iterations=350,
    betta_template_init_meetings=0.5,
    nest_shrink_prop=0.2,
)

print("-------------------------")
print("Ant colony initialized")
print("-------------------------")

print("-------------------------")
print("similarity Dictionary length")
print("-------------------------")
print(len(ant_clust.saved_similaritys))

# -----------------------------------
#       find Clusters
# -----------------------------------
print("finding cluters")
time_find_clusters = time.time()
ant_clust.find_clusters()
time_find_clusters = time.time() - time_find_clusters
print("")
print("Time to find Clusters")
print("------------------------------")
print(time_find_clusters)
print()

antclust_cluster_labels = ant_clust.get_clusters()

making similarity function vector
Make Ant Colony
-------------------------
Ant colony initialized
-------------------------
-------------------------
similarity Dictionary length
-------------------------
157201
finding cluters
AntClust: phase 1 of 3 -> meeting ants
left meetings  184625 / 184625
left meetings  147700 / 184625
left meetings  110775 / 184625
left meetings  73850 / 184625
left meetings  36925 / 184625
AntClust: phase 2 of 3 -> shrink nests
AntClust: phase 3 of 3 -> reassign ants

Time to find Clusters
------------------------------
10.223198413848877



In [7]:
antclust_cluster_labels

array([ 0,  0,  0, ..., 23, 11, 11])

# kmeans

In [11]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, rand_score

kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(data_np)
kmeans_cluster_labels = kmeans.labels_



In [12]:
print('score antclust:', adjusted_rand_score(labels, antclust_cluster_labels))
print('score kmeans:', adjusted_rand_score(labels, kmeans_cluster_labels))

score antclust: 0.01437743536889416
score kmeans: -0.026019614142710475


# Testing the individual clusters

In [7]:
# normal numeric Distance, dataset is normalized so we know min = 0 max = 1
min = 0
max = 1
sim_numeric = similarity_1d(min, max)

num_features = len(data_np[0])
print("making similarity function vector")
similarity_per_feature = [sim_numeric] * num_features

# -----------------------------------
#       define ruleset
# -----------------------------------
ruleset = labroche_rules()

# -----------------------------------
#       define AntClust
# -----------------------------------
print("Make Ant Colony")
ant_clust = AntClust(
    data_np,
    similarity_per_feature,
    ruleset,
    store_computed_similaritys=True,
    alpha_ant_meeting_iterations=150,
    betta_template_init_meetings=0.5,
    nest_shrink_prop=0.2,
)

print("-------------------------")
print("Ant colony initialized")
print("-------------------------")

print("-------------------------")
print("similarity Dictionary length")
print("-------------------------")
print(len(ant_clust.saved_similaritys))

# -----------------------------------
#       find Clusters
# -----------------------------------
print("finding cluters")
time_find_clusters = time.time()
ant_clust.find_clusters()
time_find_clusters = time.time() - time_find_clusters
print("")
print("Time to find Clusters")
print("------------------------------")
print(time_find_clusters)
print()

print("List of Labels:")
for i in ruleset.labels:
    print(i)

# hack for printing and visualize Clusters
print("labels with ants and how many ants")
label_dic = {}
for ant in ant_clust.ants:
    if ant.label in label_dic:
        label_dic[ant.label] += 1
    else:
        label_dic[ant.label] = 1

print("Clusters found: ", len(label_dic))
print(label_dic)

print("Sorted dictionary")
print("-----------------------------")
dic2 = dict(sorted(label_dic.items(), key=lambda x: x[1]))
print(dic2)

making similarity function vector
Make Ant Colony
-------------------------
Ant colony initialized
-------------------------
-------------------------
similarity Dictionary length
-------------------------
73791
finding cluters
AntClust: phase 1 of 3 -> meeting ants
left meetings  79125 / 79125
left meetings  63300 / 79125
left meetings  47475 / 79125
left meetings  31650 / 79125
left meetings  15825 / 79125
AntClust: phase 2 of 3 -> shrink nests
AntClust: phase 3 of 3 -> reassign ants

Time to find Clusters
------------------------------
13.005173206329346

List of Labels:
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
1